In [1]:
from keras.models import load_model
import coremltools
import pickle
import os
#from tensorflow.keras.applications.mobilenet import MobileNet
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
#from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
#import keras
import sklearn
#import tensorflow as tf

In [19]:
print(keras.__version__)

2.2.4


In [20]:
print(sklearn.__version__)

0.19.2


In [21]:
print(tf.__version__)

2.3.1


In [4]:
model_path = os.path.sep.join(['output4', 'animal.model'])
le_path = os.path.sep.join(['spider', 'lb.pickle'])

In [7]:
# load the class labels
print("[INFO] loading class labels from label binarizer")
lb = pickle.loads(open(le_path, "rb").read())
print(lb.classes_)
class_labels = lb.classes_.tolist()
print("[INFO] class labels: {}".format(class_labels))


[INFO] loading class labels from label binarizer
['Australia_Redback_Spider' 'Australia_White-Tailed_Spider'
 'Australian_Garden_Orb_Weaver_Spider' 'Australian_Tarantula_Spider'
 'Sydney_Funnel_Web_Spider']
[INFO] class labels: ['Australia_Redback_Spider', 'Australia_White-Tailed_Spider', 'Australian_Garden_Orb_Weaver_Spider', 'Australian_Tarantula_Spider', 'Sydney_Funnel_Web_Spider']


In [24]:
# load the trained convolutional neural network
print("[INFO] loading model...")
model = load_model(model_path)
print(model)

[INFO] loading model...


In [5]:
input_tensor = Input(shape=(224,224,3))
base_model = MobileNetV2(
            include_top=True,
            weights='imagenet',
            input_tensor=input_tensor,
            input_shape=(224,224,3),
            pooling='avg')

In [26]:
coreml_model = coremltools.convert(model,
                                                   input_names=['image'],
                                                   image_input_names="image",
                                                   image_scale=1/255.0,
                                                   class_labels = class_labels,
                                                   is_bgr=True)

Translating MIL ==> MLModel Ops: 100%|██████████| 753/753 [00:00<00:00, 3086.12 ops/s]


In [27]:
print(coreml_model)

input {
  name: "input_1"
  type {
    multiArrayType {
      shape: 1
      shape: 224
      shape: 224
      shape: 3
      dataType: FLOAT32
      shapeRange {
        sizeRanges {
          lowerBound: 1
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 224
          upperBound: 224
        }
        sizeRanges {
          lowerBound: 224
          upperBound: 224
        }
        sizeRanges {
          lowerBound: 3
          upperBound: 3
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.3.1"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}



In [28]:
output = model_path.rsplit('.', 1)[0] + '.mlmodel'
coreml_model.save(output)